#Intro

In [1]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.

In [2]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.transforms.core import DoFn

In [3]:
!mkdir -p data

In [4]:
from google.colab import files
uploaded = files.upload()

Saving 005_dept_data.txt to 005_dept_data.txt


In [5]:
import pandas as pd

columns = ['EmployeeID', 'Name', 'DepartmentID', 'Department', 'StartDate']

df = pd.read_csv('005_dept_data.txt', header=None, names=columns, delimiter=',')
df.head()

,EmployeeID,Name,DepartmentID,Department,StartDate
0,149633CM,Marco,10,Accounts,1-01-2019
1,212539MU,Rebekah,10,Accounts,1-01-2019
2,231555ZZ,Itoe,10,Accounts,1-01-2019
3,503996WI,Edouard,10,Accounts,1-01-2019
4,704275DC,Kyle,10,Accounts,1-01-2019


In [6]:
df.describe()

,DepartmentID
count,898.000000
mean,20.356347
std,8.088555
min,10.000000
25%,10.000000
50%,20.000000
75%,30.000000
max,30.000000


#Code

In [9]:
def SplitRow(element):
    return element.split(',')

p = beam.Pipeline()


input_collection = (
                      p
                      | "Read from text file" >> beam.io.ReadFromText('005_dept_data.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )

accounts_count = (
                    input_collection
                    | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
                    | 'Pair each accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " +record[1], 1))
                    | 'Group and sum1' >> beam.CombinePerKey(sum)
                    | 'Write results for account' >> beam.io.WriteToText('data/Account')
                 )

hr_count = (
                    input_collection
                    | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
                    | 'Pair each hr employee with 1' >> beam.Map(lambda record: ("HR, " +record[1], 1))
                    | 'Group and sum' >> beam.CombinePerKey(sum)
                    | 'Write results for hr' >> beam.io.WriteToText('data/HR')
           )

output =(
    (accounts_count,hr_count)
    | beam.Flatten()
    | beam.io.WriteToText('data/both')
)



p.run()

In [8]:
!{('head -n 20 /content/data/both-00000-of-00001')}

('Accounts, Marco', 31)
('Accounts, Rebekah', 31)
('Accounts, Itoe', 31)
('Accounts, Edouard', 31)
('Accounts, Kyle', 62)
('Accounts, Kumiko', 31)
('Accounts, Gaston', 31)
('Accounts, Ayumi', 30)
('HR, Beryl', 62)
('HR, Olga', 31)
('HR, Leslie', 31)
('HR, Mindy', 31)
('HR, Vicky', 31)
('HR, Richard', 31)
('HR, Kirk', 31)
('HR, Kaori', 31)
('HR, Oscar', 31)


In [11]:
!{('head -n 20 /content/data/Account-00000-of-00001')}

('Accounts, Marco', 31)
('Accounts, Rebekah', 31)
('Accounts, Itoe', 31)
('Accounts, Edouard', 31)
('Accounts, Kyle', 62)
('Accounts, Kumiko', 31)
('Accounts, Gaston', 31)
('Accounts, Ayumi', 30)


In [10]:
!{('head -n 20 /content/data/HR-00000-of-00001')}

('HR, Beryl', 62)
('HR, Olga', 31)
('HR, Leslie', 31)
('HR, Mindy', 31)
('HR, Vicky', 31)
('HR, Richard', 31)
('HR, Kirk', 31)
('HR, Kaori', 31)
('HR, Oscar', 31)


#word_count

In [28]:
def run():
    input_file = '/content/data.txt'
    output_file = 'data/wordcount_output_1'

    with beam.Pipeline() as p:
        (
            p
            | "Read File" >> beam.io.ReadFromText(input_file)  # Odczyt danych z pliku
            | "Split into Words" >> beam.FlatMap(lambda line: line.split())  # Rozdzielenie wierszy na słowa
            | "Pair with 1" >> beam.Map(lambda word: (word, 1))  # Mapowanie każdego słowa na parę (słowo, 1)
            | "Count Words" >> beam.CombinePerKey(sum)  # Sumowanie wartości dla każdego słowa
            | "Format Results" >> beam.Map(lambda word_count: f"word: {word_count[0]} count: {word_count[1]}")  # Formatowanie wyników
            | "Write Results" >> beam.io.WriteToText(output_file, file_name_suffix='.txt')  # Zapis wyników do pliku
        )

# Uruchomienie funkcji
if __name__ == "__main__":
    run()

In [29]:
!{('head -n 20 /content/data/wordcount_output_1-00000-of-00001.txt')}

word: KING count: 242
word: LEAR count: 222
word: DRAMATIS count: 1
word: PERSONAE count: 1
word: king count: 29
word: of count: 439
word: Britain count: 1
word: (KING count: 1
word: LEAR:) count: 1
word: OF count: 15
word: FRANCE: count: 1
word: DUKE count: 3
word: BURGUNDY count: 6
word: (BURGUNDY:) count: 1
word: CORNWALL count: 56
word: (CORNWALL:) count: 1
word: ALBANY count: 60
word: (ALBANY:) count: 1
word: EARL count: 2
word: KENT count: 136


In [30]:
import re

input_file = '/content/data.txt'
output_file = 'data/wordcount_output_2'

with beam.Pipeline() as p:
  (
      p
      | 'Read lines' >> beam.io.ReadFromText(input_file)
      | 'Find words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z']+", line))
      | 'Pair words with 1' >> beam.Map(lambda word: (word, 1))
      | 'Group and sum' >> beam.CombinePerKey(sum)
    #   | 'Format results' >> beam.Map(lambda word_count: str(word_count))
      | "Format Results" >> beam.Map(lambda word_count: f"word: {word_count[0]} count: {word_count[1]}")
      | 'Write results' >> beam.io.WriteToText(output_file, file_name_suffix='.txt')

  )

In [31]:
!{('head -n 20 /content/data/wordcount_output_2-00000-of-00001.txt')}

word: KING count: 243
word: LEAR count: 236
word: DRAMATIS count: 1
word: PERSONAE count: 1
word: king count: 65
word: of count: 447
word: Britain count: 2
word: OF count: 15
word: FRANCE count: 10
word: DUKE count: 3
word: BURGUNDY count: 8
word: CORNWALL count: 63
word: ALBANY count: 67
word: EARL count: 2
word: KENT count: 156
word: GLOUCESTER count: 141
word: EDGAR count: 126
word: son count: 29
word: to count: 438
word: Gloucester count: 26
